In [1]:
import torch
from WeaponData_resnet50 import WeaponData
from torchvision import models
import torch.nn as nn
from torch.nn import functional as F

In [2]:
dataset = WeaponData()

['banknot', 'karta płatnicza', 'nóż', 'pistolet', 'portfel', 'smartphone']


In [3]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True).cuda()
model.fc = nn.Sequential(
               nn.Linear(2048, 512),
               nn.ReLU(inplace=True),
               nn.Linear(512, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 32),
               nn.ReLU(inplace=True),
               nn.Linear(32, 6)).cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters())

In [4]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
                data = dataset.trainloader
            else:
                model.eval()
                data = dataset.testloader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data:
                inputs = inputs.cuda()
                labels = labels.cuda()

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data)
            epoch_acc = running_corrects.double() / len(data)

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [5]:
model_trained = train_model(model, criterion, optimizer, num_epochs=50 )
torch.save(model_trained, "./output_data/models/weapon_resnet50")

Epoch 1/50
----------
train loss: 9.5308, acc: 11.5941
validation loss: 4.9254, acc: 13.2109
Epoch 2/50
----------
train loss: 5.5968, acc: 13.0235
validation loss: 4.2543, acc: 13.4688
Epoch 3/50
----------
train loss: 5.0369, acc: 13.2941
validation loss: 4.0436, acc: 13.5547
Epoch 4/50
----------
train loss: 4.1816, acc: 13.5490
validation loss: 3.8066, acc: 13.6797
Epoch 5/50
----------
train loss: 4.3199, acc: 13.5294
validation loss: 4.4544, acc: 13.3906
Epoch 6/50
----------
train loss: 3.9713, acc: 13.5902
validation loss: 3.0214, acc: 13.8906
Epoch 7/50
----------
train loss: 3.6514, acc: 13.6706
validation loss: 3.0614, acc: 13.8438
Epoch 8/50
----------
train loss: 3.4106, acc: 13.8137
validation loss: 3.1092, acc: 13.8750
Epoch 9/50
----------
train loss: 3.4350, acc: 13.7843
validation loss: 2.9357, acc: 13.8984
Epoch 10/50
----------
train loss: 3.3641, acc: 13.8667
validation loss: 4.5735, acc: 13.5703
Epoch 11/50
----------
train loss: 3.0805, acc: 13.9098
validation lo